<font color="#CA0032"><h1 align="left">**Redes Generativas Adversariales (GANs)**</h1></font>

<font color="#6E6E6E"><h1 align="left">**Creación de imágenes nuevas con GANs Convolucionales Profundas (DC-GANs)**</h1></font>

<h2 align="left">Manuel Sánchez-Montañés</h2>

<font color="#6E6E6E"><h2 align="left">manuel.smontanes@gmail.com</h2></font>

In [ ]:
COLAB                  = True
SAVE_INTERMEDIATE_DATA = False

In [ ]:
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers import LeakyReLU
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import backend as K
from keras import initializers

np.random.seed(1000)

%matplotlib inline

In [ ]:
# Tamaño del espacio latente
randomDim = 20

# Carga de datos
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.astype(np.float32) / 255 # para que esté entre 0 y 1
X_train = 2*X_train - 1 # para que esté entre -1 y 1
print(X_train.shape)

X_train = X_train.reshape(X_train.shape + (1,))
print(X_train.shape)

X_train = X_train[y_train>=5] # para simplificar se entrena sólo con las clases 5,6,7,8,9
y_train = y_train[y_train>=5] # ídem

In [ ]:
X_train.shape

**Optimizadores**

In [ ]:
# optimizador para el generador:
adam_gen  = Adam(lr=0.0002, beta_1=0.5) # lr por defecto: 0.001

# optimizador para el discriminador:
adam_disc = Adam(lr=0.0002/2, beta_1=0.5)

**Red generadora ("generator")**

In [ ]:
randomDim

In [ ]:
# UpSampling2D
# Toma:

#  1 5 3
#  8 9 2
#  0 1 0

# Devuelve:

#  1 1 5 5 3 3
#  1 1 5 5 3 3
#  8 8 9 9 2 2
#  8 8 9 9 2 2
#  0 0 1 1 0 0
#  0 0 1 1 0 0

In [ ]:
generator = Sequential()

# completar

generator.summary()

**Discriminador**

In [ ]:
from keras.layers import MaxPool2D

In [ ]:
X_train.shape

In [ ]:
discriminator = Sequential()

# completar

discriminator.summary()

**Red combinada (sistema GAN)**

In [ ]:
# completar



gan.summary()

In [ ]:
# Plot the loss from each batch
def plotLoss(epoch):
    plt.figure(figsize=(10, 3))
    plt.plot(range(1,len(dLosses)+1), dLosses,
             label='Discriminitive loss', linewidth=3)
    plt.plot(range(1,len(gLosses)+1), gLosses,
             label='Generative loss', linewidth=3)
    plt.xlabel('Epoch', fontsize=16)
    plt.ylabel('Loss', fontsize=16)
    plt.legend(fontsize=14)
    if not COLAB:
        plt.savefig('./images/dcgan_loss_epoch_{}.png'.format(epoch))
    plt.show()

# Create a wall of generated images
def plotGeneratedImages(epoch, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, randomDim])
    generatedImages = generator.predict(noise)

    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
#        plt.imshow(generatedImages[i, 0], interpolation='nearest', cmap='gray_r')
        plt.imshow(generatedImages[i,:,:,0], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    if SAVE_INTERMEDIATE_DATA:
        plt.savefig('./images/dcgan_generated_image_epoch_{}.png'.format(epoch))
    plt.show()

    
def plotImages(images, nrows, ncols, figsize):
    plt.figure(figsize=figsize)
    for i in range(images.shape[0]):
        plt.subplot(nrows, ncols, i+1)
        plt.imshow(images[i,:,:,0], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.show()
    

# Save the generator and discriminator networks (and weights) for later use
def saveModels(epoch):
    generator.save('models/dcgan_generator_epoch_{}.h5'.format(epoch))
    discriminator.save('models/dcgan_discriminator_epoch_{}.h5'.format(epoch))

In [ ]:
if SAVE_INTERMEDIATE_DATA:
    os.makedirs("./images", exist_ok=True)
    os.makedirs("./models", exist_ok=True)

In [ ]:
dLosses = [] # histórico de los valores de la función de coste del discriminador
gLosses = [] # histórico de los valores de la función de coste del generador

In [ ]:
epochs=50
batchSize=128
epocas_refrescar_grafica=1

batchCount = len(X_train) // batchSize
print('Epochs:', epochs)
print('Batch size:', batchSize)
print('Batches per epoch:', batchCount)

for e in range(1, epochs+1):
    print('-'*15, 'Epoch %d' % e, '-'*15)
    for _ in tqdm(range(batchCount)):

        # ** EMPIEZA MINI-ENTRENAMIENTO DISCRIMINADOR: **
        
        # Genero entrada aleatoria al generador para batchSize (128) imágenes:
        noise = np.random.normal(0, 1, size=[batchSize, randomDim])

        # Genero imágenes falsas a través del generator:
        generatedImages = generator.predict(noise)

        # Selecciono al azar batchSize (128) imágenes reales
        imageBatch = X_train[np.random.randint(0, len(X_train), size=batchSize)]
        
        # Genero un X donde las 128 primeras imágenes son reales y las 128 siguientes fake
        X = np.concatenate([imageBatch, generatedImages])
        
        # Genero las etiquetas para estas 128+128 imágenes:
        # 128 "casi unos" (clase "real") seguidos de 128 "casi ceros" (clase "fake")
        yDis = np.array(batchSize*[0.9] + batchSize*[0.1])
        
        # Descongelo el discriminador:
        discriminator.trainable = True

        # Entreno discriminador
        dloss = discriminator.train_on_batch(X, yDis)

        # ** TERMINA MINI-ENTRENAMIENTO DISCRIMINADOR **


        # ** EMPIEZA MINI-ENTRENAMIENTO GENERADOR: **
        
        # Genero randomDim variables latentes (ruido) de entrada al generador
        # por cada una de las batchSize imágenes que quiero generar:
        noise = np.random.normal(0, 1, size=[batchSize, randomDim])

        # Genero etiquetas que deseo que el discriminador genere al pasarle
        # las imágenes creadas por el generador (deseo engañarle, con lo que
        # la salida deseada es 0.9, "casi real")
        yGen = np.array(batchSize*[0.9])

        # Congelo el discriminador (en este paso solo aprende el generador):
        discriminator.trainable = False

        # Entreno el sistema (en realidad solo se entrena el generador ya que
        # he congelado el discriminador):
        gloss = gan.train_on_batch(noise, yGen)

        # ** TERMINA MINI-ENTRENAMIENTO GENERADOR **
        
        
    # Store loss of most recent batch from this epoch
    dLosses.append(dloss)
    gLosses.append(gloss)
    
    if (e==1) or ((e%epocas_refrescar_grafica)==0):
        plotGeneratedImages(e)
        if SAVE_INTERMEDIATE_DATA:
            saveModels(e)
    if (e%epocas_refrescar_grafica)==0:
        plotLoss(e)

In [ ]:
# Plot losses from every epoch
plotLoss(e)

In [ ]:
# Generación de 100 imágenes nuevas:

n_imagenes = 100

noise = np.random.normal(0, 1, size=[n_imagenes, randomDim])
generatedImages = generator.predict(noise)
plotGeneratedImages(e+1)

Ahora generamos un conjunto de vectores de entrada a la GAN. Cada vector de entrada tiene **randomDim** componentes:

In [ ]:
nrows = 5
ncols = 10
input0 = np.zeros((nrows*ncols, randomDim))
caso = 0
for i in range(nrows):
    for j in range(ncols):
        input_id = i+10
        input0[caso,input_id] = -2+4*j/(ncols-1)
        caso = caso + 1

In [ ]:
# Vamos a mostrar los resultados obtenidos para el conjunto de vectores
# de entrada en una matriz de nfilas * ncols:
nfilas = 5
ncols  = 10

# Inicializo a 0 el conjunto de vectores de entrada a la GAN:
input0 = np.zeros((nfilas*ncols, randomDim))

# Termino de calcular el conjunto de vectores de entrada.
# La idea es que en cada fila las componentes diferentes de cero
# son las mismas, y sus valores cambian de columna a columna:

nvector = 0
for i in range(nfilas):
    # Qué componentes de las randomDim se van a perturbar:
    componentes_pert = range(15+i,15+i+1)
    for j,x in enumerate(np.linspace(-2, 2, ncols)):
        input_id = i+0
        input0[nvector][componentes_pert] = x
        nvector = nvector + 1

In [ ]:
generatedImages = generator.predict(input0)
plotImages(generatedImages, nfilas, ncols, figsize=(14,7))

In [ ]:
generatedImages.shape

In [ ]:
# Para grabar las redes a fichero:

generator.save("./dcgan_generator.h5")
generator.save_weights("./dcgan_generator_weights.h5")
discriminator.save("./dcgan_discriminator.h5")
discriminator.save_weights("./dcgan_discriminator_weights.h5")
gan.save("./dcgan_gan.h5")
gan.save_weights("./dcgan_gan_weights.h5")